In [1]:
Dataset_url = "https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia"

In [2]:
# Upload Kaggle json file
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
 99% 2.27G/2.29G [00:09<00:00, 241MB/s]
100% 2.29G/2.29G [00:09<00:00, 249MB/s]


In [4]:
!unzip -q chest-xray-pneumonia.zip

In [5]:
!pip install -q timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 61.0 MB/s eta 0:00:00


In [13]:
import timm
from fastai.vision.all import *
import os


In [14]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cuda'

In [15]:
# Define paths to your train, validation, and test data
train_path = Path('/content/chest_xray/train')
val_path = Path('/content/chest_xray/val')
test_path = Path('/content/chest_xray/test')

In [16]:
!ls

chest_xray  chest-xray-pneumonia.zip  kaggle.json  sample_data


In [17]:
!mv chest_xray/val/NORMAL/* chest_xray/train/NORMAL/

In [18]:
!mv chest_xray/val/PNEUMONIA/* chest_xray/train/PNEUMONIA/

In [19]:
data = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms()
)

In [20]:
# Get DataLoaders from the datablock
dls = data.dataloaders(train_path)

In [21]:
timm.list_models("resnet*")

['resnet10t',
 'resnet14t',
 'resnet18',
 'resnet18d',
 'resnet26',
 'resnet26d',
 'resnet26t',
 'resnet32ts',
 'resnet33ts',
 'resnet34',
 'resnet34d',
 'resnet50',
 'resnet50_clip',
 'resnet50_clip_gap',
 'resnet50_gn',
 'resnet50_mlp',
 'resnet50c',
 'resnet50d',
 'resnet50s',
 'resnet50t',
 'resnet50x4_clip',
 'resnet50x4_clip_gap',
 'resnet50x16_clip',
 'resnet50x16_clip_gap',
 'resnet50x64_clip',
 'resnet50x64_clip_gap',
 'resnet51q',
 'resnet61q',
 'resnet101',
 'resnet101_clip',
 'resnet101_clip_gap',
 'resnet101c',
 'resnet101d',
 'resnet101s',
 'resnet152',
 'resnet152c',
 'resnet152d',
 'resnet152s',
 'resnet200',
 'resnet200d',
 'resnetaa34d',
 'resnetaa50',
 'resnetaa50d',
 'resnetaa101d',
 'resnetblur18',
 'resnetblur50',
 'resnetblur50d',
 'resnetblur101d',
 'resnetrs50',
 'resnetrs101',
 'resnetrs152',
 'resnetrs200',
 'resnetrs270',
 'resnetrs350',
 'resnetrs420',
 'resnetv2_50',
 'resnetv2_50d',
 'resnetv2_50d_evos',
 'resnetv2_50d_frn',
 'resnetv2_50d_gn',
 'resnetv2

In [22]:
# Load ResNet18 model without pre-trained weights using Timm
model = timm.create_model('resnetv2_50', pretrained=False, num_classes=dls.c)

In [24]:
# Create a Fastai Learner
learn = Learner(dls, model, metrics=accuracy)

In [25]:
dls.c

2

In [26]:
# Train the model
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,time
0,0.338760,11.821469,0.246654,01:35
1,0.322503,14.928752,0.256214,01:30
2,0.278694,1.417655,0.278203,01:30
3,0.240359,0.332275,0.891013,01:32
4,0.218866,0.456783,0.772467,01:31
5,0.204460,7.728442,0.277247,01:33
6,0.190721,0.402189,0.814532,01:32
7,0.167660,0.137208,0.948375,01:31
8,0.152526,0.140684,0.946463,01:31
9,0.139992,0.136427,0.945507,01:30


In [27]:
# Load the test data
test_files = get_image_files(test_path)

In [28]:
# Create a DataLoader for the test set
test_dl = dls.test_dl(test_files, with_labels=True)  # Set with_labels=False if test set has no labels

In [29]:
# Validate the model on the test set
results = learn.validate(dl=test_dl)

In [30]:
# Print results
print(results)

[0.21081043779850006, 0.932692289352417]


In [31]:
# Print formatted results
print(f"Loss: {results[0]:.4f}, Accuracy: {results[1] * 100:.2f}%")

Loss: 0.2108, Accuracy: 93.27%
